# Our own implementation of the LU Decomposition

In [4]:
import numpy as np

In [5]:
def my_lu(A):
    """
    Implement the LU Decomposition of A.
    
    This function returns a tuple of two matrices L and U,
    that together form the LU Decomposition of A such that
    A = LU.
    """
    
    # Return an error if the matrix is not square
    if not A.shape[0] == A.shape[1]:
        raise ValueError("Input must be square.")
        
    n = A.shape[0]
    
    L = np.zeros((n, n), dtype=np.float64)
    U = np.zeros((n, n), dtype=np.float64)
    p = np.arange(n)
    U[:] = A
    np.fill_diagonal(L, 1)
    
    for i in range(n - 1):
        max_index = i + np.argmax(np.abs(U[i:, i]))
        U[[i, max_index], :] = U[[max_index, i], :]
        L[[i, max_index], :i] = L[[max_index, i], :i]
        p[[i, max_index]] = p[[max_index, i]]
        for row in range(i + 1, n):
            L[row, i] = U[row, i] / U[i, i]
            U[row, i:] = U[row, i:] - L[row, i] * U[i, i:]
            U[row, i] = 0
            
    P = np.eye(n)[p, :]
    return P, L, U

In [8]:
n = 500

rand = np.random.RandomState(0)

A = rand.randn(n, n)

P, L, U = my_lu(A)

residual = np.linalg.norm(P @ A - L @ U, np.inf) / np.linalg.norm(A, np.inf)
print(residual)

4.9433747022116635e-15


In [11]:
from scipy.linalg import lu, lu_factor, lu_solve

In [13]:
lu, piv = lu_factor(A)

In [15]:
b = np.random.randn(n)
x = lu_solve((lu, piv), b)

In [40]:
from scipy.linalg import lu

n = 500
A = rand.randn(n, n)

%timeit my_lu(A)
%timeit lu(A)



428 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
34.7 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
from scipy.linalg import solve_triangular

b = rand.randn(n)

x = solve_triangular(U,
        solve_triangular(L, b, lower=True),
                    )

In [46]:
relative_residual = np.linalg.norm(b - A @ x) / (np.linalg.norm(A) * np.linalg.norm(x))
print(relative_residual)

1.8457353823417516e-13


In [47]:
x = np.linalg.solve(A, b)

In [49]:
from scipy.linalg import lu_factor
help(lu_factor)

Help on function lu_factor in module scipy.linalg.decomp_lu:

lu_factor(a, overwrite_a=False, check_finite=True)
    Compute pivoted LU decomposition of a matrix.
    
    The decomposition is::
    
        A = P L U
    
    where P is a permutation matrix, L lower triangular with unit
    diagonal elements, and U upper triangular.
    
    Parameters
    ----------
    a : (M, M) array_like
        Matrix to decompose
    overwrite_a : bool, optional
        Whether to overwrite data in A (may increase performance)
    check_finite : bool, optional
        Whether to check that the input matrix contains only finite numbers.
        Disabling may give a performance gain, but may result in problems
        (crashes, non-termination) if the inputs do contain infinities or NaNs.
    
    Returns
    -------
    lu : (N, N) ndarray
        Matrix containing U in its upper triangle, and L in its lower triangle.
        The unit diagonal elements of L are not stored.
    piv : (N,) ndarray